# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Effects of Electrostatics on Silica Oligomeriz...,Talk,talk-1,AIChE 2020 Annual Meeting,01/11/2020,Virtual,NaN,NaN
1,Dimensionality Reduction of Reaction Coordinat...,Talk,talk-2,"Catalysis and Computation Symposium, Universit...",01/09/2019,"Bristol, UK",NaN,NaN
2,Dimensionality Reduction of Pathways Generated...,Poster,poster-1,CECAM Workshop: Learning the Collective Variab...,01/07/2019,"Paris, France",NaN,NaN
3,Dimensionality Reduction of Chemical Reaction ...,Talk,talk-3,Physical Organic Gordon Research Conference (GRC,01/06/2019,"Holderness, NH",NaN,NaN
4,Dimensionality Reduction of Chemical Reaction ...,Poster,poster-2,Physical Organic Gordon Research Seminar (GRS)...,01/06/2019,"Holderness, NH",NaN,NaN
5,Dimensionality Reduction of Chemical Reaction ...,Poster,poster-3,"Applied Mathematics: The Next 50 Years, Univer...",01/06/2019,"Seattle, WA",NaN,NaN
6,Dimensionality Reduction of Reaction Coordinat...,Talk,talk-4,Spring ACS National Meeting & Exposition,01/03/2019,"Orlando, FL",NaN,NaN
7,Women Chemists Committee (WCC) Science of Sexu...,Symposium,symposium-1,Spring ACS National Meeting & Exposition,01/03/2018,"New Orleans, LA",NaN,NaN
8,Pummerer-Like Rearrangement: Effects of Solven...,Poster,poster-4,Spring ACS National Meeting & Exposition: Chem...,01/03/2018,"New Orleans, LA",NaN,NaN
9,Control factors involved in abietadiene synthe...,Talk,talk-5,Fall ACS National Meeting & Exposition: Women ...,01/08/2017,"Washington, DC",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2014-poster-13.md   2016-poster-9.md    2017-talk-6.md      2019-poster-3.md
2015-poster-11.md   2016-talk-7.md      2018-poster-4.md    2019-talk-2.md
2015-poster-12.md   2017-poster-5.md    2018-symposium-1.md 2019-talk-3.md
2015-talk-8.md      2017-poster-6.md    2018-talk-5.md      2019-talk-4.md
2016-poster-10.md   2017-poster-7.md    2019-poster-1.md    2020-talk-1.md
2016-poster-8.md    2017-talk-5.md      2019-poster-2.md    talkmap.ipynb


In [7]:
!cat ../_talks/2018-talk-5.md

---
title: "Pummerer-Like Rearrangement: Effects of Solvent on a Post-Transition State Bifurcation"
collection: talks
type: "Talk"
permalink: /talks/01/03/2018-talk-5
venue: "Spring ACS National Meeting & Exposition"
date: 01/03/2018
location: "New Orleans, LA"
---
